### Análise de Polaridade
(comentários fb)

#### 1. <span style="color:#00b3b3">Instalação de pacotes</span>

In [ ]:
if(!require('dplyr')) install.packages('dplyr')
if(!require('purr')) install.packages('purr')
if(!require('tidytext')) install.packages('tidytext')
if(!require('ggplot2')) install.packages('ggplot2')
if(!require('lexiconPT')) install.packages('lexiconPT')
if(!require('wordcloud')) install.packages('wordcloud')
source("http://bioconductor.org/biocLite.R")
biocLite("limma")

#### 2. <span style="color:#00b3b3">Bibliotecas</span>

In [ ]:
install.packages("tm", repos="http://R-Forge.R-project.org")
library(tm)

In [ ]:
library(dplyr)
library(purrr)
library(tidytext)
library(gutenbergr)
library(ggplot2)
library(lexiconPT)

library(readr)
library(stringr)
library(tidyr)
library(reshape2)
# library(syuzhet)
library(ggthemes)
library(wordcloud)
library(limma)

#### 3. <span style="color:#00b3b3">Carrega Dados</span>

In [ ]:
comments <- read.csv(file="comentarios.csv", header=TRUE, sep=",")
# comments = read.columns("comentarios.csv", c("comment_message"), sep = ",")
# comments
colnames(comments)
lapply(comments, head)

#### 4. <span style="color:#00b3b3">Pré-processamento</span>

In [ ]:
custom_stop_words <- bind_rows(stop_words,
                               tibble(word = tm::stopwords("portuguese"),
                                          lexicon = "custom"))
custom_stop_words

In [ ]:
comments %>% map_if(is.factor, as.character) %>% as_data_frame -> comments
tidy_comments <- comments %>%
    unnest_tokens(word, comment_message) %>%
  anti_join(custom_stop_words)

In [ ]:
bing_word_counts <- tidy_comments %>%
  inner_join(get_sentiments("bing")) %>%
  count(word, sentiment, sort = TRUE) %>%
  ungroup()

In [ ]:
tidy_comments %>%
  count(word, sort = TRUE)

In [ ]:
tidy_comments %>%
  count(word, sort = TRUE) %>%
  filter(n > 600) %>%
  mutate(word = reorder(word, n)) %>%
  ggplot(aes(word, n)) +
  geom_col() +
  xlab(NULL) +
  coord_flip() + ggtitle("The Most Common Words in Comments")

In [ ]:
bing_word_counts <- tidy_comments %>%
  inner_join(get_sentiments("bing")) %>%
  count(word, sentiment, sort = TRUE) %>%
  ungroup()

**4.** Polaridade

In [ ]:
bing_word_counts %>%
  group_by(sentiment) %>%
  top_n(10) %>%
  ungroup() %>%
  mutate(word = reorder(word, n)) %>%
  ggplot(aes(word, n, fill = sentiment)) +
  geom_col(show.legend = FALSE) +
  facet_wrap(~sentiment, scales = "free_y") +
  labs(y = "Words Contribute to sentiment",
       x = NULL) +
  coord_flip()

In [ ]:
tidy_comments%>%
  anti_join(stop_words) %>%
  count(word) %>%
  with(wordcloud(word, n, max.words = 100))

In [ ]:
tidy_comments %>%
  inner_join(get_sentiments("bing")) %>%
  count(word, sentiment, sort = TRUE) %>%
  acast(word ~ sentiment, value.var = "n", fill = 0) %>%
  comparison.cloud(colors = c("#F8766D", "#00BFC4"),
                   max.words = 100)